In [1]:
import numpy as np 
import pandas as pd
import tensorflow as tf 



In [7]:
from tensorflow.keras.models import load_model

In [13]:
model = load_model('models\saracasm_class')

In [14]:
import re

def decontractions(phrase):
    """decontracted takes text and convert contractions into natural form.
     ref: https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python/47091490#47091490"""
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"won\’t", "will not", phrase)
    phrase = re.sub(r"can\’t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)

    phrase = re.sub(r"n\’t", " not", phrase)
    phrase = re.sub(r"\’re", " are", phrase)
    phrase = re.sub(r"\’s", " is", phrase)
    phrase = re.sub(r"\’d", " would", phrase)
    phrase = re.sub(r"\’ll", " will", phrase)
    phrase = re.sub(r"\’t", " not", phrase)
    phrase = re.sub(r"\’ve", " have", phrase)
    phrase = re.sub(r"\’m", " am", phrase)

    return phrase

In [19]:
def df_to_dataset(dataframe, shuffle=True, batch_size=1024):
    df = dataframe.copy()
    labels = df.pop('label')
    df = df['comment']
    ds = tf.data.Dataset.from_tensor_slices((df, labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds

In [16]:
def predict_single_string(model, encoder, input_string, max_length):
    # Encode the input string using the provided encoder
    input_encoded = encoder(np.array([input_string]))
    # Pad the encoded sequence to match the model input shape
    input_padded = tf.keras.preprocessing.sequence.pad_sequences(input_encoded, maxlen=max_length, padding='post')
    # Make prediction
    prediction = model.predict(input_padded)
    return prediction

In [10]:
string = "This is defininitly not a string"

string = decontractions(string)

In [52]:
testing_string = "I enjoyed that movie. The plot was engaging, and the acting was superb."

# Create a DataFrame with a 'comment' column
test_df = pd.DataFrame({'comment': [testing_string],
                        'label':[0]})

# Call the df_to_dataset function
test_dataset = df_to_dataset(test_df, shuffle=False)  # Shuffle should be False for testing

# Now, you can use the test_dataset for testing your model

In [53]:
encoder = tf.keras.layers.TextVectorization(max_tokens=2000) 
encoder.adapt(test_dataset.map(lambda text,label:text))

In [55]:
predictions = model.predict(test_dataset)

1/1 [==============================] - 0s 8ms/step


In [60]:
predictions[0][0]

0.37374273